<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/examples/callbacks/OpenInferenceCallback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenInference Callback Handler + Arize Phoenix

[OpenInference](https://github.com/Arize-ai/open-inference-spec) is an open standard for capturing and storing AI model inferences. It enables production LLMapp servers to seamlessly integrate with LLM observability solutions such as [Arize](https://arize.com/) and [Phoenix](https://github.com/Arize-ai/phoenix).

The `OpenInferenceCallbackHandler` saves data from LLM applications for downstream analysis and debugging. In particular, it saves the following data in columnar format:

- query IDs
- query text
- query embeddings
- scores (e.g., cosine similarity)
- retrieved document IDs

This tutorial demonstrates the callback handler's use for both in-notebook experimentation and lightweight production logging.

⚠️ The `OpenInferenceCallbackHandler` is in beta and its APIs are subject to change.

ℹ️ If you find that your particular query engine or use-case is not supported, open an issue on [GitHub](https://github.com/Arize-ai/open-inference-spec/issues).

## Install Dependencies and Import Libraries

Install notebook dependencies.

In [ ]:
!pip install -q html2text llama-index pandas tqdm

Import libraries.


In [ ]:
import hashlib
import json
from pathlib import Path
import os
import textwrap
from typing import List, Union

from llama_index import (
    SimpleWebPageReader,
    ServiceContext,
    VectorStoreIndex,
)
from llama_index.callbacks import CallbackManager, OpenInferenceCallbackHandler
from llama_index.callbacks.open_inference_callback import (
    as_dataframe,
    QueryData,
    NodeData,
)
from llama_index.node_parser import SimpleNodeParser
import pandas as pd
from tqdm import tqdm

## Load and Parse Documents

Load documents from Paul Graham's essay "What I Worked On".

In [ ]:
documents = SimpleWebPageReader().load_data(
    [
        "http://raw.githubusercontent.com/jerryjliu/llama_index/main/examples/paul_graham_essay/data/paul_graham_essay.txt"
    ]
)
print(documents[0].text)

		

What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.

The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in

Parse the document into nodes. Display the first node's text.

In [ ]:
parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)
print(nodes[0].text)

What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.

The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in the

## Access Data as a Pandas Dataframe

When experimenting with chatbots and LLMapps in a notebook, it's often useful to run your chatbot against a small collection of user queries and collect and analyze the data for iterative improvement. The `OpenInferenceCallbackHandler` stores your data in columnar format and provides convenient access to the data as a pandas dataframe.

Instantiate the OpenInference callback handler and attach to the service context.

In [ ]:
callback_handler = OpenInferenceCallbackHandler()
callback_manager = CallbackManager([callback_handler])
service_context = ServiceContext.from_defaults(
    callback_manager=callback_manager
)

Build the index and instantiate the query engine.

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)
query_engine = index.as_query_engine()

Run your query engine across a collection of queries.

In [ ]:
max_characters_per_line = 80
queries = [
    "What did Paul Graham do growing up?",
    "When and how did Paul Graham's mother die?",
    "What, in Paul Graham's opinion, is the most distinctive thing about YC?",
    "When and how did Paul Graham meet Jessica Livingston?",
    "What is Bel, and when and where was it written?",
]
for query in queries:
    response = query_engine.query(query)
    print("Query")
    print("=====")
    print(textwrap.fill(query, max_characters_per_line))
    print()
    print("Response")
    print("========")
    print(textwrap.fill(str(response), max_characters_per_line))
    print()

Query
=====
What did Paul Graham do growing up?

Response
 Paul Graham grew up writing short stories and programming on an IBM 1401. He
eventually convinced his father to buy him a TRS-80, and he wrote simple games,
a program to predict how high his model rockets would fly, and a word processor.
He went to college to study philosophy, but found it boring and switched to AI.
He wrote essays and published them online, and eventually wrote a book called
Hackers & Painters. He also worked on spam filters, painted, and cooked for
groups of friends.

Query
=====
When and how did Paul Graham's mother die?

Response
 Paul Graham's mother died on January 15, 2014. The cause of death was a stroke
caused by a blood clot caused by colon cancer. Paul Graham had been visiting her
regularly and had been focusing on her care since her cancer had returned.

Query
=====
What, in Paul Graham's opinion, is the most distinctive thing about YC?

Response
 The most distinctive thing about YC, according to Pa

The data from your query engine runs can be accessed as a pandas dataframe for analysis and iterative improvement.

In [ ]:
query_data_buffer = callback_handler.flush_query_data_buffer()
query_dataframe = as_dataframe(query_data_buffer)
query_dataframe

:id.id:        :timestamp.iso_8601:  \
0  0f5ee89f-bd19-474b-b610-1baa19eff39a  2023-07-25T01:10:38.552887   
1  fe25c43f-2a4d-4413-af75-8ba0f73478a1  2023-07-25T01:10:45.532510   
2  68ed8155-b0dc-4742-a1d2-e2434026ef18  2023-07-25T01:10:47.877096   
3  cd9d9940-feac-47d8-be28-9c018c749e2e  2023-07-25T01:10:54.647328   
4  4f5ba9ea-cc45-43f3-87ab-980e8c4fb1fc  2023-07-25T01:11:01.614051   

                                :feature.text:prompt  \
0                What did Paul Graham do growing up?   
1         When and how did Paul Graham's mother die?   
2  What, in Paul Graham's opinion, is the most di...   
3  When and how did Paul Graham meet Jessica Livi...   
4    What is Bel, and when and where was it written?   

                   :feature.[float].embedding:prompt  \
0  [0.007235619239509106, -0.009616627357900143, ...   
1  [0.015601863153278828, 0.004369843751192093, -...   
2  [0.002805074444040656, 0.0014766178792342544, ...   
3  [0.002249377314001322, -0.002420470817014575, ...   
4  [0.009005682542920113, -0.013750467449426651, ...   

                           :prediction.text:response  \
0  \nPaul Graham grew up writing short stories an...   
1  \nPaul Graham's mother died on January 15, 201...   
2  \nThe most distinctive thing about YC, accordi...   
3  \nPaul Graham met Jessica Livingston at a part...   
4  \nBel is a new Lisp language that was written ...   

        :feature.[str].retrieved_document_ids:prompt  \
0  [b4d0b960aa09e693f9dc0d50ef46a3d0bf5a8fb3ac9f3...   
1  [087de1ac941068615e4892676e879e7ddbb7e31d1746c...   
2  [0faaf9fd3067ce0a7d1ad5092c168b482db8ff787a612...   
3  [3f85a689b50d492e5189934a36d48819c99fa126579ae...   
4  [00ea608f27c4dffb04d5d332f494cda668774ff1f3ede...   

  :feature.[float].retrieved_document_scores:prompt  
0           [0.795533397271937, 0.7874512413922382]  
1          [0.7753400850918623, 0.7529468329757473]  
2          [0.8433254678997132, 0.8340653419635877]  
3          [0.8040675911753389, 0.7924490041096467]  
4          [0.7948174831996746, 0.7635244754576934]

The dataframe column names conform to the OpenInference spec, which specifies the category, data type, and intent of each column.

## Log Production Data

In a production setting, LlamaIndex application maintainers can log the data generated by their system by implementing and passing a custom `callback` to `OpenInferenceCallbackHandler`. The callback is of type `Callable[List[QueryData]]` that accepts a buffer of query data from the `OpenInferenceCallbackHandler`, persists the data (e.g., by uploading to cloud storage or sending to a data ingestion service), and flushes the buffer after data is persisted. A reference implementation is included below that periodically writes data in OpenInference format to local Parquet files when the buffer exceeds a certain size.

In [ ]:
class ParquetCallback:
    def __init__(
        self, data_path: Union[str, Path], max_buffer_length: int = 1000
    ):
        self._data_path = Path(data_path)
        self._data_path.mkdir(parents=True, exist_ok=False)
        self._max_buffer_length = max_buffer_length
        self._batch_index = 0

    def __call__(
        self,
        query_data_buffer: List[QueryData],
        node_data_buffer: List[NodeData],
    ) -> None:
        if len(query_data_buffer) > self._max_buffer_length:
            query_dataframe = as_dataframe(query_data_buffer)
            file_path = self._data_path / f"log-{self._batch_index}.parquet"
            query_dataframe.to_parquet(file_path)
            self._batch_index += 1
            query_data_buffer.clear()  # ⚠️ clear the buffer or it will keep growing forever!
            node_data_buffer.clear()  # didn't log node_data_buffer, but still need to clear it

⚠️ In a production setting, it's important to clear the buffer, otherwise, the callback handler will indefinitely accumulate data in memory and eventually cause your system to crash.

Attach the Parquet writer to your callback and re-run the query engine. The data will be saved to disk.

In [ ]:
data_path = "data"
parquet_writer = ParquetCallback(
    data_path=data_path,
    # this parameter is set artificially low for demonstration purposes
    # to force a flush to disk, in practice it would be much larger
    max_buffer_length=1,
)
callback_handler = OpenInferenceCallbackHandler(callback=parquet_writer)
callback_manager = CallbackManager([callback_handler])
service_context = ServiceContext.from_defaults(
    callback_manager=callback_manager
)
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)
query_engine = index.as_query_engine()

for query in tqdm(queries):
    query_engine.query(query)

100%|██████████| 5/5 [00:19<00:00,  3.86s/it]


Load and display saved Parquet data from disk to verify that the logger is working. 

In [ ]:
query_dataframes = []
for file_name in os.listdir(data_path):
    file_path = os.path.join(data_path, file_name)
    query_dataframes.append(pd.read_parquet(file_path))
query_dataframe = pd.concat(query_dataframes)
query_dataframe

:id.id:        :timestamp.iso_8601:  \
0  d6dd4544-3488-4e31-9b65-e01fe27b5763  2023-07-25T01:11:33.563546   
1  30046fe6-7b89-42a8-89ba-471a9bcb248b  2023-07-25T01:11:39.481920   
0  afe899eb-d42f-4971-8529-bad5833143bf  2023-07-25T01:11:27.630485   
1  a4375367-e889-4c70-a749-2a22a8d8c321  2023-07-25T01:11:30.678282   

                                :feature.text:prompt  \
0  What, in Paul Graham's opinion, is the most di...   
1  When and how did Paul Graham meet Jessica Livi...   
0                What did Paul Graham do growing up?   
1         When and how did Paul Graham's mother die?   

                   :feature.[float].embedding:prompt  \
0  [0.002805074444040656, 0.0014766178792342544, ...   
1  [0.002249377314001322, -0.002420470817014575, ...   
0  [0.007235619239509106, -0.009616627357900143, ...   
1  [0.015666792169213295, 0.004303410183638334, -...   

                           :prediction.text:response  \
0  \nThe most distinctive thing about YC, accordi...   
1  \nPaul Graham met Jessica Livingston at a part...   
0  \nPaul Graham grew up writing short stories an...   
1  \nPaul Graham's mother died on January 15, 201...   

        :feature.[str].retrieved_document_ids:prompt  \
0  [0faaf9fd3067ce0a7d1ad5092c168b482db8ff787a612...   
1  [3f85a689b50d492e5189934a36d48819c99fa126579ae...   
0  [b4d0b960aa09e693f9dc0d50ef46a3d0bf5a8fb3ac9f3...   
1  [087de1ac941068615e4892676e879e7ddbb7e31d1746c...   

  :feature.[float].retrieved_document_scores:prompt  
0          [0.8433254678997132, 0.8340653419635877]  
1          [0.8040675911753389, 0.7924490041096467]  
0           [0.795533397271937, 0.7874512413922382]  
1           [0.7753338483836093, 0.752899027860476]